In [1]:
import cv2 as cv

In [2]:
BLUR_RADIUS = 21
erode_kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
dilate_kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (9, 9))

In [ ]:
cap = cv.VideoCapture(0)

# Capture several frames to allow the camera's autoexposure to adjust.
for _ in range(10):
    ret, frame = cap.read()
    if not ret:
        exit(1)
    gray_background = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray_background = cv.GaussianBlur(gray_background, (BLUR_RADIUS, BLUR_RADIUS), 0)

ret, frame = cap.read()

while ret:
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray_frame = cv.GaussianBlur(gray_frame, (BLUR_RADIUS, BLUR_RADIUS), 0)

    diff = cv.absdiff(gray_background, gray_frame)
    _, thresh = cv.threshold(diff, 40, 255, cv.THRESH_BINARY)
    cv.erode(thresh, erode_kernel, thresh, iterations=2)
    cv.dilate(thresh, dilate_kernel, thresh, iterations=2)

    contours, hier = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    for c in contours:
        if cv.contourArea(c) > 4000:
            x, y, w, h = cv.boundingRect(c)
            cv.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

    cv.imshow('diff', diff)
    cv.imshow('thresh', thresh)
    cv.imshow('detection', frame)

    k = cv.waitKey(1)
    if k == 27: # Escape
        break
cap.release()
cv.destroyAllWindows()